In [37]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [38]:
! mkdir encoder
! curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

A subdirectory or file encoder already exists.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  146M    0  122k    0     0   124k      0  0:20:10 --:--:--  0:20:10  125k
 15  146M   15 22.5M    0     0  11.3M      0  0:00:12  0:00:01  0:00:11 11.4M
 39  146M   39 57.8M    0     0  19.4M      0  0:00:07  0:00:02  0:00:05 19.4M
 66  146M   66 97.8M    0     0  24.5M      0  0:00:05  0:00:03  0:00:02 24.5M
 97  146M   97  143M    0     0  28.7M      0  0:00:05  0:00:04  0:00:01 28.8M
100  146M  100  146M    0     0  29.0M      0  0:00:05  0:00:05 --:--:-- 35.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0

In [54]:
import pandas as pd

texts = []
#with open ("/content/sts2016-english-with-gs-v1.0/STS2016.input.answer-answer.txt", 'rt',encoding="utf8") as myfile:  
#with open ("/content/sts2016-english-with-gs-v1.0/STS2016.input.headlines.txt", 'rt',encoding="utf8") as myfile:  
#with open ("/content/sts2016-english-with-gs-v1.0/STS2016.input.postediting.txt", 'rt',encoding="utf8") as myfile:  
#with open ("/content/sts2016-english-with-gs-v1.0/STS2016.input.question-question.txt", 'rt',encoding="utf8") as myfile:  
with open ("/content/sts2016-english-with-gs-v1.0/STS2016.input.plagiarism.txt", 'rt',encoding="utf8") as myfile:
    for myline in myfile:
        texts.append(myline)

sentence_list = [x for i in texts for x in i.split("\t")]

out = [sentence_list[i: i+4] for i in range(0, len(sentence_list), 4)]

df_inferSent = pd.DataFrame(out, columns = ['Sentence 1', 'Sentence 2', 'Link 1', 'Link 2']) 

In [40]:
model_name = 'bert-base-nli-mean-tokens'

In [41]:
from sentence_transformers import SentenceTransformer

In [42]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VaishuSistas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
model_1 = SentenceTransformer(model_name)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
 # Load model for INfersent Model version 1 - Uses GLove and Model 2 uses Fasttext
from models import InferSent
model_version = 1 # Use model_version = 2 to use FastText
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [47]:
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model

In [48]:
# Provide Location of the Glove Folder and FastText Folder
W2V_PATH = 'content/GloVe/glove.840B.300d.txt' if model_version == 1 else 'content/fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [49]:
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [20]:
# Load some sentences
sentences_1 = df_inferSent["Sentence 1"].tolist()
sentences_2 = df_inferSent["Sentence 2"].tolist()


In [21]:
embeddings_1 = model.encode(df_inferSent["Sentence 1"].tolist(),True)
embeddings_2 = model.encode(df_inferSent["Sentence 2"].tolist(),True)
print('No of Sentences encoded : {0}'.format(len(embeddings_1)))
#embeddings_2

C:\Users\VaishuSistas\NLP_Assignment1\models.py:209: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


No of Sentences encoded : 1271


In [22]:
embeddings_1

array([[ 0.06604282,  0.15754834,  0.0279735 , ..., -0.01615116,
         0.02378302,  0.02222021],
       [ 0.07242656,  0.13110851,  0.10929569, ...,  0.05319538,
         0.00388105, -0.00504598],
       [ 0.08463684,  0.10130055,  0.08161053, ...,  0.02132878,
        -0.02551951, -0.01072006],
       ...,
       [ 0.05539126,  0.07839144,  0.15074709, ...,  0.01010915,
        -0.03814262,  0.07881027],
       [ 0.08834228,  0.07716344,  0.07161196, ...,  0.04279687,
         0.05160971,  0.01084205],
       [ 0.0257999 ,  0.10578486,  0.09493136, ...,  0.07119673,
         0.05455206,  0.00246286]], dtype=float32)

In [23]:
embeddings_2

array([[ 0.06604282,  0.09777914,  0.13704017, ...,  0.01322717,
        -0.02033604,  0.04833248],
       [ 0.048799  ,  0.13501674,  0.09684775, ...,  0.05319538,
         0.00388105,  0.00172025],
       [ 0.06195145,  0.08290235,  0.10114814, ...,  0.00102582,
        -0.03814263, -0.02826982],
       ...,
       [ 0.02783349,  0.08085911,  0.06413528, ...,  0.0878832 ,
        -0.03814263,  0.04944415],
       [ 0.08834228,  0.11747027,  0.0901776 , ..., -0.03270331,
         0.05550285,  0.00248738],
       [ 0.04268726,  0.142569  ,  0.00243938, ...,  0.02243408,
        -0.00399751, -0.02064569]], dtype=float32)

In [24]:
 def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [25]:
inferSentScore = []
for i in range(len(embeddings_1)):
     inferSentScore.append(cosine(embeddings_1[i], embeddings_2[i]))

In [26]:
inferSentScore

[0.78207725,
 0.9442197,
 0.7576871,
 0.74270165,
 0.97843444,
 0.8198177,
 0.8723637,
 0.90620685,
 0.9594361,
 0.8083832,
 0.72362155,
 0.80327797,
 0.846386,
 0.8708072,
 0.7843873,
 0.72332364,
 0.81690764,
 0.7485928,
 0.8873513,
 0.9014598,
 0.9828185,
 0.92209005,
 0.7269015,
 0.9585082,
 0.74177754,
 0.79015493,
 0.97364527,
 0.99691105,
 0.82170236,
 0.8529432,
 0.77374005,
 0.82708627,
 0.8997009,
 0.78677887,
 0.778528,
 0.74861586,
 0.9300975,
 0.83051646,
 0.80560416,
 0.6775533,
 0.9464692,
 0.75747824,
 0.7748853,
 0.8197215,
 0.6911861,
 0.7227638,
 0.69292253,
 0.9436661,
 0.8073457,
 0.7552227,
 0.7650748,
 0.77269506,
 0.79016536,
 0.7806908,
 0.9549885,
 0.863458,
 0.70659655,
 0.83261436,
 0.8411937,
 0.8062628,
 0.80336595,
 0.84210587,
 0.8141296,
 0.8767294,
 0.87830454,
 0.692722,
 0.94122434,
 0.8195178,
 0.76819295,
 0.82687396,
 0.71201277,
 0.8097927,
 0.7750244,
 0.7726819,
 0.8567573,
 0.79519016,
 0.834343,
 0.65157485,
 0.7736971,
 0.8137951,
 0.8745184

In [27]:
# Scaling the score to calculate the Pearson's
inferSentScore_5 = [x* 5 for x in inferSentScore]

In [28]:

inferSentScore_5

[3.9103862643241882,
 4.72109854221344,
 3.788435459136963,
 3.7135082483291626,
 4.89217221736908,
 4.099088609218597,
 4.361818432807922,
 4.531034231185913,
 4.797180593013763,
 4.041916131973267,
 3.6181077361106873,
 4.016389846801758,
 4.231930077075958,
 4.3540358543396,
 3.9219364523887634,
 3.6166182160377502,
 4.084538221359253,
 3.742963969707489,
 4.436756372451782,
 4.507299065589905,
 4.914092421531677,
 4.610450267791748,
 3.6345073580741882,
 4.792540967464447,
 3.7088876962661743,
 3.950774669647217,
 4.86822634935379,
 4.984555244445801,
 4.108511805534363,
 4.264715909957886,
 3.8687002658843994,
 4.135431349277496,
 4.498504400253296,
 3.9338943362236023,
 3.8926398754119873,
 3.7430793046951294,
 4.65048760175705,
 4.152582287788391,
 4.028020799160004,
 3.387766480445862,
 4.732345938682556,
 3.787391185760498,
 3.874426484107971,
 4.09860759973526,
 3.4559303522109985,
 3.613818883895874,
 3.464612662792206,
 4.718330502510071,
 4.0367284417152405,
 3.77611339092

In [29]:
df_inferSent.insert( loc=2 , column = "Raw Score" , value = inferSentScore)
df_inferSent.insert( loc=3 , column = "Scaled Score" , value = inferSentScore_5)

In [30]:
del df_inferSent['Link 1']
del df_inferSent['Link 2']

In [31]:
df_inferSent

,Sentence 1,Sentence 2,Raw Score,Scaled Score
0,The vector space model has some limitations: 1.,The vector space model are the documents which...,0.782077,3.910386
1,Secondly to define the value of the optimal so...,Define value of optimal solution recursively.,0.944220,4.721099
2,Vector space representation results in the los...,"If a term occurs in the document, the value wi...",0.757687,3.788435
3,P(B) is the prior or marginal probability of B...,"P(A), or the probability that the student is a...",0.742702,3.713508
4,"It is ""prior"" in the sense that it does not ta...","It is ""previous"" in the sense that it does not...",0.978434,4.892172
...,...,...,...,...
1266,Every dimension is precisely related to a sepa...,Each dimension corresponds to a separate term.,0.843468,4.217342
1267,"As a formal theorem, Bayes' theorem is valid i...",Bayes' Theorem is a theorem of probability the...,0.805279,4.026397
1268,This means that inheritance is used when types...,It does not take into account any information ...,0.686267,3.431334
1269,It is used in information retrieval and was fi...,"It is used in information filtering, indexing,...",0.777779,3.888896


In [32]:
df_inferSent['Scaled Score'].max()

4.984555244445801

In [33]:
infersent_maxScoreSentences = df_inferSent['Scaled Score'].values == df_inferSent['Scaled Score'].max()
df_inferSent.loc[infersent_maxScoreSentences]

,Sentence 1,Sentence 2,Raw Score,Scaled Score
27,"Thus, the ""program"" is the optimal plan for ac...","Therefore, the ""program"" is the optimal plan f...",0.996911,4.984555


In [34]:
infersent_minScoreSentences = df_inferSent['Scaled Score'].values == df_inferSent['Scaled Score'].min()
df_inferSent.loc[infersent_minScoreSentences]

,Sentence 1,Sentence 2,Raw Score,Scaled Score
463,This is highly used in dynamic programming.,There are four steps in Dynamic Programming: 1.,0.561434,2.807169


In [35]:
df_inferSent.to_csv(r'STS2016.input.plagiarism.txt.csv', index = False)